In [32]:
import numpy as np
from scipy.stats import t
import pandas as pd

### Confidence Interval Formula

* $ \hat{y_0} \pm t_{crit}\cdot s.e.$
* $ s.e. = \sqrt{MS_{Res} \cdot (X_0^T(X^TX)^{-1}X_0)}$
* $X_0 = [x_1, \cdots x_n]$

### Prediction Interval Formula

* $ \hat{y_0} \pm t_{crit}\cdot \sqrt{MS_{Res} \cdot (1+(X_0^T(X^TX)^{-1}X_0))}$

### reference site
* https://www.real-statistics.com/multiple-regression/confidence-and-prediction-intervals/
* https://stats.stackexchange.com/questions/147242/how-to-calculate-the-prediction-interval-for-an-ols-multiple-regression/147254


In [33]:
df=pd.read_excel('sample.xlsx')

In [34]:
def ci_pi(x0, df, y, t_a=0.975):
    X = df.drop(y,1)
    y = df[y]
    
    dfRes = X.shape[0] - X.shape[1] - 1
    
    X_col = X.columns.tolist()
    X['constant'] = 1 # intercept를 위함
    X_f = X[['constant']+X_col].copy()
    
        
    # 공분산 역행렬, 회귀계수 도출 후 예측값 구하기
    cov = np.linalg.inv(np.matmul(X_f.values.T, X_f.values))
    coef = np.matmul(np.matmul(cov, X_f.values.T), y)
    y0 = np.matmul(x0, coef.reshape(-1,1))    

    
    # 잔차 통계량 구하기
    res = y.values - np.matmul(X_f.values, coef.reshape(-1, 1)).reshape(-1, )
    msRes = np.sum(res ** 2) / dfRes
    
    # t 함수값 구하기
    t_ = t(dfRes)
    t_res = t_.ppf(t_a)
    
    # 정해진 식에 따라 표준오차 구하기
    var_ci = np.sqrt(msRes* np.matmul(np.matmul(x0, cov), x0.T))
    var_pi = np.sqrt(msRes* np.matmul(np.matmul(x0, cov), x0.T) + msRes)
    
    # 최종 식에 따른 신뢰구간 도출
    ci_low, ci_upp = y0 - (var_ci*t_res), y0+(var_ci*t_res)
    pi_low, pi_upp = y0 - (var_pi*t_res), y0+(var_pi*t_res)
    
    return ci_low[0][0], ci_upp[0][0], pi_low[0][0], pi_upp[0][0]

In [35]:
x0 = np.array([[1, 7, 80, 400]])
ci_pi(x0, df.iloc[:, 1:], 'Poverty', 0.975)

(12.144995207917642, 13.590365544716288, 7.842878385248262, 17.892482367385668)